# NYC Health COVID-19 Data Scraper
### March 30, 2020
### Matthew J. Beattie
### University of Oklahoma

Cases obtained from _https://www1.nyc.gov/assets/doh/downloads/pdf/imm/covid-19-daily-data-summary.pdf_

Deaths obtained from _https://www1.nyc.gov/assets/doh/downloads/pdf/imm/covid-19-daily-data-summary-deaths.pdf_

In [154]:
import pandas as pd
import re
import PyPDF2 as pdf
import tempfile
import urllib.request
from datetime import date

yr = 2020
mo = 4
day = 3

## Define Common Routines

In [155]:
"""
textfrompdf()
Reads in PDF file from NYC website or local file.  To read a local file, the user
must flag the readfrom variable as 'local'
"""
def textfrompdf(url,readfrom='online'):
    if readfrom != 'local':
        dataIn = urllib.request.urlopen(url).read()
    else:
        localf = open(url, 'rb')
        dataIn = localf.read()
        localf.close()

    fp = tempfile.TemporaryFile()

    # Write the pdf data to a temp file
    fp.write(dataIn)

    pdfReader = pdf.PdfFileReader(fp)
    pdfReader.getNumPages()

    # Get the first page
    return pdfReader.getPage(0).extractText()

## Read in NYC Cases Data

In [156]:
# Read in text from the PDF file and display
url = 'https://www1.nyc.gov/assets/doh/downloads/pdf/imm/covid-19-daily-data-summary-04032020-2.pdf'
localurl = 'covid-19-daily-data-summary.pdf'
casespage = textfrompdf(url,readfrom='online')

casespage

'Coronavirus Disease 2019 (COVID\n-19) \n  Daily \nData\n Summary\n  The data in this report reflect events and activities\n as of\n April \n3, 2020\n at 4:30 P\nM.  All data in this report are preliminary and subject to change as cases continue to be investigated. \n These\n data \ninclude \ncases in NYC residents and foreign residents treated in NYC facilities\n.  NYC COVID\n-19 Cases\n .  Total Cases\n Total\n 56289 Median Age (Range)\n 49 (0\n-107) Age Group\n  -  0 to 17\n 988 (2%)\n -  18 to 44\n 22434 (40%)\n -  45 to 64\n 19971 (36%)\n -  65 to 74\n 7064 (13%)\n -  75 and over\n 5709 (10%)\n -  Unknown\n 123 Age 50 and over\n  -  Yes\n 27939 (50%)\n -  No 28227 (50%)\n Sex\n  -  Female\n 25536 (45%)\n -  Male\n 30672 (55%)\n -  Unknown\n 81 Borough\n  -  Bronx\n 10765 (19%)\n -  Brooklyn\n 15327 (27%)\n -  Manhattan\n 8222 (15%)\n -  Queens\n 18823 (33%)\n -  Staten Island\n 3117 (6%)\n -  Unknown\n 35 Deaths\n 1867         \n \n \n  '

## Extract the COVID cases by Age

In [157]:
# Extract the values for the cases by Age.  Ignore Unknown Age. 
tblText = re.sub("Coronavirus.*Cases.*0 to 17", "0 to 17", casespage, flags=re.DOTALL)
tblText = re.sub("Deaths.*","", tblText, flags=re.DOTALL)
tblText = re.sub("Unknown.*", "", tblText, flags=re.DOTALL)
tblText = re.sub("\(\d+%", "", tblText, flags=re.DOTALL)
tblText = re.sub(" \).+?(?=[a-zA-Z0-9])", ";", tblText, flags=re.DOTALL)
tblText = re.sub("\n.+?(?=[0-9])", ",", tblText, flags=re.DOTALL)
tblText = re.sub(" \).+", "", tblText, flags=re.DOTALL)
tblText = re.sub("0 to 17", "0-17", tblText, flags=re.DOTALL)
tblText = re.sub("18 to 44", "18-44", tblText, flags=re.DOTALL)
tblText = re.sub("45 to 64", "45-64", tblText, flags=re.DOTALL)
tblText = re.sub("65 to 74", "65-74", tblText, flags=re.DOTALL)
tblText = re.sub("75 and over", "Over_75", tblText, flags=re.DOTALL)


In [158]:
tblText

'0-17,988;18-44,22434;45-64,19971;65-74,7064;Over_75,5709'

In [159]:
# Split the extracted text into rows and remove any rows that do not contain data
rows = re.split(";", tblText)
rows = [row for row in rows if re.search(r"[0-9]",row)!=None]

In [160]:
rows

['0-17,988', '18-44,22434', '45-64,19971', '65-74,7064', 'Over_75,5709']

In [161]:
# Convert the rows of text into a list of lists
agedata = []
for row in rows:
    record = []
    field1 = re.search("^([^,])+", row).group()
    field2 = re.search("[^,]+$", row).group()
    record.append(field1)
    record.append(int(field2))
    agedata.append(record)

In [162]:
# Convert the list into a dataframe and display 
agedf = pd.DataFrame(agedata, columns=['category','count'])
agedf['feature']='age'
agedf['type']='cases'
agedf['date']=date(yr,mo,day)
agedf

,category,count,feature,type,date
0,0-17,988,age,cases,2020-04-03
1,18-44,22434,age,cases,2020-04-03
2,45-64,19971,age,cases,2020-04-03
3,65-74,7064,age,cases,2020-04-03
4,Over_75,5709,age,cases,2020-04-03


## Extract the COVID cases by Sex

In [163]:
# Extract the values for the cases by Sex.  Ignore Unknown Sex. 
tblText = re.sub("Coronavirus.*Cases.*Female", "Female", casespage, flags=re.DOTALL)
tblText = re.sub("Deaths.*","", tblText, flags=re.DOTALL)
tblText = re.sub("Unknown.*", "", tblText, flags=re.DOTALL)
tblText = re.sub("\(\d+%", "", tblText, flags=re.DOTALL)
tblText = re.sub(" \).+?(?=[a-zA-Z0-9])", ";", tblText, flags=re.DOTALL)
tblText = re.sub("\n.+?(?=[0-9])", ",", tblText, flags=re.DOTALL)
tblText = re.sub(" \).+", "", tblText, flags=re.DOTALL)

In [164]:
# Split the extracted text into rows and remove any rows that do not contain data
rows = re.split(";", tblText)
rows = [row for row in rows if re.search(r"[0-9]",row)!=None]

In [165]:
# Convert the rows into a list of lists
sexdata = []
for row in rows:
    record = []
    field1 = re.search("^([^,])+", row).group()
    field2 = re.search("[^,]+$", row).group()
    record.append(field1)
    record.append(int(field2))
    sexdata.append(record)

# Convert the list into a dataframe and display
sexdf = pd.DataFrame(sexdata, columns=['category','count'])
sexdf['feature']='sex'
sexdf['type']='cases'
sexdf['date']=date(yr,mo,day)
sexdf

,category,count,feature,type,date
0,Female,25536,sex,cases,2020-04-03
1,Male,30672,sex,cases,2020-04-03


## Extract the COVID cases by Borough

In [166]:
# Extract the values for the cases by Borough.  Ignore Unknown Borough. 
tblText = re.sub("Coronavirus.*Cases.*Bronx", "Bronx", casespage, flags=re.DOTALL)
tblText = re.sub("Deaths.*","", tblText, flags=re.DOTALL)
tblText = re.sub("Unknown.*", "", tblText, flags=re.DOTALL)
tblText = re.sub("\(\d+%", "", tblText, flags=re.DOTALL)
tblText = re.sub(" \).+?(?=[a-zA-Z0-9])", ";", tblText, flags=re.DOTALL)
tblText = re.sub("\n.+?(?=[0-9])", ",", tblText, flags=re.DOTALL)
tblText = re.sub(" \).+", "", tblText, flags=re.DOTALL)
tblText = re.sub("Staten Island", "Staten_Island", tblText, flags=re.DOTALL)

# Split the extracted text into rows and remove any rows that do not contain data
rows = re.split(";", tblText)
rows = [row for row in rows if re.search(r"[0-9]",row)!=None]

In [167]:
# Covert the rows of text into a list of lists
boroughdata = []
for row in rows:
    record = []
    field1 = re.search("^([^,])+", row).group()
    field2 = re.search("[^,]+$", row).group()
    record.append(field1)
    record.append(int(field2))
    boroughdata.append(record)

# Convert the list into a dataframe and display    
boroughdf = pd.DataFrame(boroughdata, columns=['category','count'])
boroughdf['feature']='borough'
boroughdf['type']='cases'
boroughdf['date']=date(yr,mo,day)

# Concatenate Age, Sex, and Borough Stats into a cases dataframe and display
casesdf = pd.concat([agedf, sexdf, boroughdf]).reset_index().drop(columns=['index'])
casesdf

,category,count,feature,type,date
0,0-17,988,age,cases,2020-04-03
1,18-44,22434,age,cases,2020-04-03
2,45-64,19971,age,cases,2020-04-03
3,65-74,7064,age,cases,2020-04-03
4,Over_75,5709,age,cases,2020-04-03
5,Female,25536,sex,cases,2020-04-03
6,Male,30672,sex,cases,2020-04-03
7,Bronx,10765,borough,cases,2020-04-03
8,Brooklyn,15327,borough,cases,2020-04-03
9,Manhattan,8222,borough,cases,2020-04-03


## Read in NYC Deaths Data
Note that the deaths program is a bit more efficient.  We handle the entire table in one pass instead of breaking it into Age, Sex, and Borough.

In [168]:
# Read in text from PDF and show in raw format
url = 'https://www1.nyc.gov/assets/doh/downloads/pdf/imm/covid-19-daily-data-summary-deaths-04032020-2.pdf'
localurl = 'covid-19-daily-data-summary-deaths-200330.pdf'
deathspage = textfrompdf(url,readfrom='online')
deathspage

'Coronavirus Disease 2019 (COVID\n-19) \n  Daily \nData\n Summary\n  The data in this report reflect events and \nactivities\n as of\n April 3,\n 2020\n at 4\n:30 \nPM.  All data in this report are preliminary and subject to change as cases continue to be investigated. \n These\n data \ninclude \ncases in NYC residents and foreign residents treated in NYC facilities\n.  NYC COVID\n-19 Deaths\n . Underlying \nConditions\n1 No \nUnderlying \nConditions\n Underlying \nConditions \nPending\n Total\n Age Group\n     -  0 to 17\n 1 0 0 1 -  18 to 44\n 75 8 27 110 -  45 to 64\n 347 16 79 442 -  65 to 74\n 337 1 123 461 -  75 and over\n 525 3 325 853 Sex\n     -  Female\n 490 5 211 706 -  Male\n 793 23 343 1159 -  Unknown\n 2 0 0 2 Borough\n     -  Bronx\n 419 7 54 480 -  Brooklyn\n 277 7 201 485 -  Manhattan\n 141 3 71 215 -  Queens\n 382 11 197 590 -  Staten Island\n 65 0 31 96 -  Unknown\n 1 0 0 1 Total\n 1285 28 554 1867  1Underlying \nillnesses include Diabetes, Lung Disease, Cancer, Immu

## Extract the COVID Deaths and Parse

In [169]:
# Parse the deathspage text into a set of records
tblText = re.sub("Coronavirus.*0 to 17", "0 to 17", deathspage, flags=re.DOTALL)
tblText = re.sub("Total.*","", tblText, flags=re.DOTALL)
tblText = re.sub("\n", ",", tblText, flags=re.DOTALL)
tblText = re.sub(" \-.+?(?=[a-zA-Z0-9])", ";", tblText, flags=re.DOTALL)
tblText = re.sub(" \-.+", "", tblText, flags=re.DOTALL)
tblText = re.sub(" Sex.+?(?=;)", "", tblText, flags=re.DOTALL)
tblText = re.sub(" Borough.+?(?=;)", "", tblText, flags=re.DOTALL)
tblText = re.sub("0 to 17", "0-17", tblText, flags=re.DOTALL)
tblText = re.sub("18 to 44", "18-44", tblText, flags=re.DOTALL)
tblText = re.sub("45 to 64", "45-64", tblText, flags=re.DOTALL)
tblText = re.sub("65 to 74", "65-74", tblText, flags=re.DOTALL)
tblText = re.sub("75 and over", "Over_75", tblText, flags=re.DOTALL)
tblText = re.sub(", ", ",", tblText, flags=re.DOTALL)
tblText = re.sub(" ", ",", tblText, flags=re.DOTALL)
tblText = re.sub("Staten,Island", "Staten_Island", tblText, flags=re.DOTALL)
tblText = re.sub("Unknown.+?(?=B)", "", tblText, flags=re.DOTALL)
tblText = re.sub("Unknown.*", "", tblText, flags=re.DOTALL)


In [170]:
# Display parsed text to check format
tblText

'0-17,1,0,0,1;18-44,75,8,27,110;45-64,347,16,79,442;65-74,337,1,123,461;Over_75,525,3,325,853;Female,490,5,211,706;Male,793,23,343,1159;Bronx,419,7,54,480;Brooklyn,277,7,201,485;Manhattan,141,3,71,215;Queens,382,11,197,590;Staten_Island,65,0,31,96;'

In [171]:
# Split the extracted text into rows and remove any rows that do not contain data
rows = re.split(";", tblText)
rows = [row for row in rows if re.search(r"[0-9]",row)!=None]

In [172]:
rows

['0-17,1,0,0,1',
 '18-44,75,8,27,110',
 '45-64,347,16,79,442',
 '65-74,337,1,123,461',
 'Over_75,525,3,325,853',
 'Female,490,5,211,706',
 'Male,793,23,343,1159',
 'Bronx,419,7,54,480',
 'Brooklyn,277,7,201,485',
 'Manhattan,141,3,71,215',
 'Queens,382,11,197,590',
 'Staten_Island,65,0,31,96']

In [173]:
# Convert text into a list of rows
dftemp = []
for row in rows:
    record = row.split(",")
    del record[-1]
    for i in range(1,4):
        record[i] = int(record[i])
    if (record[0][0].isdigit() or record[0][0]=='O'):
        record.append('age')
    elif (record[0]=='Female' or record[0]=='Male'):
        record.append('sex')
    else:
        record.append('borough')
    dftemp.append(record)
   

In [174]:
# Transpose list of rows into higher normal form
deathsdata = []
deathtype = ['death_underlying','death_no_underlying','death_underlying_pending']
for row in dftemp:
    for i in range(1,4):
        record = []
        record.append(row[-1])
        record.append(row[0])
        record.append(deathtype[i-1])
        record.append(row[i])
        deathsdata.append(record)
   

In [175]:
# Convert list into pandas dataframe and display
deathsdf = pd.DataFrame(deathsdata, columns=['feature','category','type','count'])
deathsdf['date']=date(yr,mo,day)
deathsdf

,feature,category,type,count,date
0,age,0-17,death_underlying,1,2020-04-03
1,age,0-17,death_no_underlying,0,2020-04-03
2,age,0-17,death_underlying_pending,0,2020-04-03
3,age,18-44,death_underlying,75,2020-04-03
4,age,18-44,death_no_underlying,8,2020-04-03
5,age,18-44,death_underlying_pending,27,2020-04-03
6,age,45-64,death_underlying,347,2020-04-03
7,age,45-64,death_no_underlying,16,2020-04-03
8,age,45-64,death_underlying_pending,79,2020-04-03
9,age,65-74,death_underlying,337,2020-04-03


In [27]:
# Concatenate cases and deaths dataframes and display
dailydf = pd.concat([casesdf, deathsdf]).reset_index().drop(columns=['index'])
dailydf

,category,count,feature,type,date
0,0-17,988,age,cases,2020-04-03
1,18-44,22434,age,cases,2020-04-03
2,45-64,19971,age,cases,2020-04-03
3,65-74,7064,age,cases,2020-04-03
4,Over_75,5709,age,cases,2020-04-03
5,Female,25536,sex,cases,2020-04-03
6,Male,30672,sex,cases,2020-04-03
7,Bronx,10765,borough,cases,2020-04-03
8,Brooklyn,15327,borough,cases,2020-04-03
9,Manhattan,8222,borough,cases,2020-04-03


In [28]:
# Save the dataframe to a CSV file
#yrstr = str(yr)
#mostr = str(mo) if mo >= 10 else ('0'+str(mo))
#daystr = str(day) if day >= 10 else ('0'+str(day))
#datestr = yrstr + mostr + daystr
#fname = datestr + '-NYCHealth-Daily-COVID-data.csv'

In [29]:
#dailydf.to_csv(fname,sep=",",index=False)

## Read in NYC Hospitalization Data, Extract and Parse

In [176]:
# Read in text from PDF and show in raw format
url = 'https://www1.nyc.gov/assets/doh/downloads/pdf/imm/covid-19-daily-data-summary-hospitalizations-04032020-1.pdf'
localurl = 'covid-19-daily-data-summary-hospitals-200330.pdf'
hosppage = textfrompdf(url,readfrom='online')
hosppage

'Coronavirus Disease 2019 (COVID\n-19) \n  Daily \nData\n Summary\n  The data in this report reflect events and activities\n as of\n April \n3, 2020\n at \n4:30\n PM.\n  All data in this report are preliminary and subject to change as cases continue to be investigated. \n These\n data \ninclude \ncases in NYC residents and foreign residents treated in NYC facilities\n.  NYC COVID\n-19 \nHospitalizations\n  . Ever \nHospitalized \nCases\n1 Total\n Cases\n Age Group\n   -  0 to 17\n 92 (9%)\n 988 -  18 to 44\n 1962 (9%)\n 22434 -  45 to 64\n 4368 (22%)\n 19971 -  65 to 74\n 2572 (36%)\n 7064 -  75 and over\n 2745 (48%)\n 5709 -  Unknown\n 0 (0%)\n 123 Sex\n   -  Female\n 4765 (19%)\n 25536 -  Male\n 6970 (23%)\n 30672 -  Unknown\n 4 (5%)\n 81 Borough\n   -  Bronx\n 2924 (27%)\n 10766 -  Brooklyn\n 2826 (18%)\n 15327 -  Manhattan\n 1414 (17%)\n 8221 -  Queens\n 3889 (21%)\n 18823 -  Staten Island\n 677 (22%)\n 3117 -  Unknown\n 9 (26%)\n 35 Total\n 11739 (21%)\n 56289  1Percentages are ro

In [177]:
# Parse the deathspage text into a set of records
tblText = re.sub("Coronavirus.*0 to 17", "0 to 17", hosppage, flags=re.DOTALL)
tblText = re.sub("Total.*","", tblText, flags=re.DOTALL)
tblText = re.sub(" \([0-9]{1,3}%\)\n ", ",", tblText, flags=re.DOTALL)
tblText = re.sub(" \-.+?(?=[a-zA-Z0-9])", ";", tblText, flags=re.DOTALL)
tblText = re.sub(" Sex.+?(?=;)", "", tblText, flags=re.DOTALL)
tblText = re.sub(" Borough.+?(?=;)", "", tblText, flags=re.DOTALL)
tblText = re.sub("0 to 17", "0-17", tblText, flags=re.DOTALL)
tblText = re.sub("18 to 44", "18-44", tblText, flags=re.DOTALL)
tblText = re.sub("45 to 64", "45-64", tblText, flags=re.DOTALL)
tblText = re.sub("65 to 74", "65-74", tblText, flags=re.DOTALL)
tblText = re.sub("75 and over", "Over_75", tblText, flags=re.DOTALL)
tblText = re.sub("\n ", ",", tblText, flags=re.DOTALL)
tblText = re.sub("Staten Island", "Staten_Island", tblText, flags=re.DOTALL)
tblText = re.sub("Unknown.+?(?=F)", "", tblText, flags=re.DOTALL)
tblText = re.sub("Unknown.+?(?=B)", "", tblText, flags=re.DOTALL)
tblText = re.sub("Unknown.*", "", tblText, flags=re.DOTALL)


In [180]:
tblText

'0-17,92,988;18-44,1962,22434;45-64,4368,19971;65-74,2572,7064;Over_75,2745,5709;Female,4765,25536;Male,6970,30672;Bronx,2924,10766;Brooklyn,2826,15327;Manhattan,1414,8221;Queens,3889,18823;Staten_Island,677,3117;'

In [181]:
# Split the extracted text into rows and remove any rows that do not contain data
rows = re.split(";", tblText)
rows = [row for row in rows if re.search(r"[0-9]",row)!=None]
rows

['0-17,92,988',
 '18-44,1962,22434',
 '45-64,4368,19971',
 '65-74,2572,7064',
 'Over_75,2745,5709',
 'Female,4765,25536',
 'Male,6970,30672',
 'Bronx,2924,10766',
 'Brooklyn,2826,15327',
 'Manhattan,1414,8221',
 'Queens,3889,18823',
 'Staten_Island,677,3117']

In [182]:
# Convert text into a list of rows
dftemp = []
for row in rows:
    record = row.split(",")
    del record[-1]
    for i in range(1,1):
        record[i] = int(record[i])
    if (record[0][0].isdigit() or record[0][0]=='O'):
        record.append('age')
    elif (record[0]=='Female' or record[0]=='Male'):
        record.append('sex')
    else:
        record.append('borough')
    dftemp.append(record)
   

In [183]:
dftemp

[['0-17', '92', 'age'],
 ['18-44', '1962', 'age'],
 ['45-64', '4368', 'age'],
 ['65-74', '2572', 'age'],
 ['Over_75', '2745', 'age'],
 ['Female', '4765', 'sex'],
 ['Male', '6970', 'sex'],
 ['Bronx', '2924', 'borough'],
 ['Brooklyn', '2826', 'borough'],
 ['Manhattan', '1414', 'borough'],
 ['Queens', '3889', 'borough'],
 ['Staten_Island', '677', 'borough']]

In [184]:
# Convert list into pandas dataframe and display
hospdf = pd.DataFrame(dftemp, columns=['category','count','feature'])
hospdf['date']=date(yr,mo,day)
hospdf['type']='hospitalizations'
hospdf

,category,count,feature,date,type
0,0-17,92,age,2020-04-03,hospitalizations
1,18-44,1962,age,2020-04-03,hospitalizations
2,45-64,4368,age,2020-04-03,hospitalizations
3,65-74,2572,age,2020-04-03,hospitalizations
4,Over_75,2745,age,2020-04-03,hospitalizations
5,Female,4765,sex,2020-04-03,hospitalizations
6,Male,6970,sex,2020-04-03,hospitalizations
7,Bronx,2924,borough,2020-04-03,hospitalizations
8,Brooklyn,2826,borough,2020-04-03,hospitalizations
9,Manhattan,1414,borough,2020-04-03,hospitalizations


In [185]:
# Concatenate cases and deaths dataframes and display
dailydf = pd.concat([casesdf, deathsdf, hospdf]).reset_index().drop(columns=['index'])
dailydf

,category,count,feature,type,date
0,0-17,988,age,cases,2020-04-03
1,18-44,22434,age,cases,2020-04-03
2,45-64,19971,age,cases,2020-04-03
3,65-74,7064,age,cases,2020-04-03
4,Over_75,5709,age,cases,2020-04-03
5,Female,25536,sex,cases,2020-04-03
6,Male,30672,sex,cases,2020-04-03
7,Bronx,10765,borough,cases,2020-04-03
8,Brooklyn,15327,borough,cases,2020-04-03
9,Manhattan,8222,borough,cases,2020-04-03


In [186]:
# Save the dataframe to a CSV file
yrstr = str(yr)
mostr = str(mo) if mo >= 10 else ('0'+str(mo))
daystr = str(day) if day >= 10 else ('0'+str(day))
datestr = yrstr + mostr + daystr
fname = datestr + '-NYCHealth-Daily-COVID-data.csv'

In [187]:
dailydf.to_csv(fname,sep=",",index=False)